<a href="https://colab.research.google.com/github/doanthanhnam1/DoanThanhNam/blob/main/cu_chi_tay_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
from os import listdir
from numpy import asarray
from numpy import save
from keras import preprocessing
from tensorflow.keras.utils import load_img, img_to_array

In [59]:
data="/content/drive/MyDrive/cu_chi/data/"

In [60]:
pixels,labels=list(),list()

for folder in listdir(data):
    if folder!='.DS_Store':
        for file in listdir(data  + folder):
            if file!='.DS_Store':
                pixel=load_img(data  + folder +"/" + file)
                pixel=img_to_array(pixel)
                pixels.append(pixel)
                labels.append( folder)

pixels =asarray(pixels)
labels =asarray(labels)#.reshape(-1,1)

In [61]:
from sklearn.preprocessing import LabelBinarizer
import pickle
from sklearn.model_selection import train_test_split
encoder = LabelBinarizer()
labels = encoder.fit_transform(labels)
#print(labels)

file = open('pix.data', 'wb')
# dump information to that file
pickle.dump((pixels,labels), file)
# close the file
file.close()

In [62]:
def load_data():
    file = open('pix.data', 'rb')
    # dump information to that file
    (pixels, labels) = pickle.load(file)
    # close the file
    file.close()
    return pixels, labels

#save_data()
X,y = load_data()
#random.shuffle(X)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=100)

In [63]:
print(X_train.shape)
print(X_test.shape)

(771, 192, 128, 3)
(193, 192, 128, 3)


In [64]:
from keras.models import Sequential
from keras.layers import Dense
from numpy.ma.core import reshape

In [65]:
X_train= X_train.astype('float32')/255
print(y_train.shape)
X_test= X_test.astype('float32')/255

(771, 4)


In [66]:
from keras import models 
from keras.models import Sequential,Model
from keras import Input
from keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D,Normalization
from keras.optimizers import Adam
batch_size=300 #lô
epochs=50     #số lần học
classes=4    #phân loại

In [67]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='linear',input_shape=(192,128,3),padding='same'))
from keras.layers import LeakyReLU
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2,2),padding='same'))
#so lan tich chap
model.add(Conv2D(64,(3,3),activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2,2),padding='same'))

In [68]:
model.add(Conv2D(128,(3,3),activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2,2),padding='same'))

In [69]:
from keras import optimizers
from keras import losses
model.add(Flatten())
model.add(Dense(1000,activation='linear'))
model.add(Dense(500,activation='linear'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(classes,activation='softmax'))
model.compile(loss=losses.categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])
model.summary()
train=model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 192, 128, 32)      896       
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 192, 128, 32)      0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 96, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 96, 64, 64)        18496     
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 96, 64, 64)        0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 48, 32, 64)       0         
 2D)                                                  

In [70]:
model.save("/content/drive/MyDrive/Colab Notebooks/BT/cu_chi.h5")